In [4]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [5]:
df = pd.read_parquet("../data/static_models/pressure_miso.par")
df = df.set_index("time")
df = df.rolling(window=40, center=True).median().dropna()
df = df.resample("5s").mean().dropna()

In [6]:
def smooth_data(df, column_name, ets_instance):
    real_time_data = df[column_name]
    smoothed_data = []
    for data_point in real_time_data:
        updated_forecast = ets_instance.update(data_point)
        smoothed_data.append(updated_forecast)
        final_forecast = ets_instance.forecast()  
    df[column_name + "_ets"] = smoothed_data

In [7]:
train_df = df["2024-01-09 17:00:00":"2024-01-10 00:30:00"]
test_df = df["2024-01-11":"2024-02-16 12:00:00"]

In [8]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from sysidentpy.basis_function._basis_function import Polynomial

In [9]:
FEATURES = "outflow"
TARGET = 'pressure'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [10]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 1,
    'order_selection': False,
    'n_terms': 3,
    'extended_least_squares': False,
    'ylag': 1,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

,Regressors,Parameters,ERR
0,y(k-1),5.3294E-01,9.9945E-01
1,x1(k-1),5.5858E-05,1.0105E-06
2,1,2.5055E-01,1.7356E-07


In [11]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

#val_plots = sys_id.residuals_analysis(yhat, df[TARGET].values)
#print(model_parameters)

RRSE: 0.5127698131275931
MSE: 0.0004246137515028898


In [12]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['pressure', 'yhat'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> pressure <i style="color:#fc9944">~53m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '76377f7a-ba2a-4e4e-abd4-22db5505ba18',
              'x': array([datetime.datetime(2024, 1, 11, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 0, 26, 25, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 0, 52, 50, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 9, 41, 55, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 0, 25, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, 45, tzinfo=<UTC>)], dtype=object),
              'y': array([0.60380001, 0.57539998, 0.59600002, ..., 0.62210649, 0.64756942,
                          0.6423611 ])},
             {'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~53m</